In [1]:
import barzur20aft
import compiler
import numpy as np
import math
import scipy
from time import time

In [2]:
def bitcoin_mdp(*args, **kwargs):
    model = barzur20aft.Bitcoin(*args, **kwargs)
    c = compiler.Compiler(model)
    return c.mdp()

In [3]:
alpha = [0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.4, 0.45, 0.49]
ptmdp = []
mdp = []
for a in alpha:
    print(a)
    m = bitcoin_mdp(alpha=a, gamma=1, maximum_fork_length=25)
    mdp.append(m)
    ptmdp.append(barzur20aft.ptmdp(m, horizon=100))

0.01
0.05
0.1
0.2
0.3
0.35
0.4
0.45
0.49


In [4]:
vi = []
for i, m in enumerate(ptmdp):
    res = m.value_iteration(value_eps=0.01)
    vi.append(res)
    v = res["value"]
    p = res["policy"]
    iter = res["iter"]
    print(
        f"alpha={alpha[i]:.2f} iter={iter:3d} v[0]={v[0]:.2f} p={p[:10]} {sum(p)} {hash(p.tobytes()) % 10000}"
    )

alpha=0.01 iter= 30 v[0]=0.14 p=[0 0 1 0 3 0 1 0 0 0] 1046 5541
alpha=0.05 iter=195 v[0]=3.23 p=[0 0 1 0 3 0 1 0 0 0] 949 1250
alpha=0.10 iter=351 v[0]=9.04 p=[0 0 1 0 3 0 1 0 0 0] 861 8136
alpha=0.20 iter=534 v[0]=22.87 p=[0 0 1 0 3 0 1 0 0 0] 771 8455
alpha=0.30 iter=664 v[0]=40.86 p=[0 0 1 0 3 0 1 0 0 0] 782 5254
alpha=0.35 iter=718 v[0]=52.17 p=[0 0 1 0 3 0 1 0 0 0] 773 4366
alpha=0.40 iter=763 v[0]=65.48 p=[0 0 1 0 3 0 1 0 0 0] 778 7242
alpha=0.45 iter=795 v[0]=80.41 p=[0 0 1 0 3 0 1 0 0 0] 733 1726
alpha=0.49 iter=804 v[0]=91.34 p=[0 0 1 0 3 0 1 0 0 0] 660 97


In [5]:
def steady_state(mdp, policy):
    n = mdp.n_states
    row = []
    col = []
    val = []

    # tutorial: https://math.stackexchange.com/a/2452452

    for src, actions in enumerate(mdp.tab):
        # markov chain itself
        for t in actions[policy[src]]:
            row.append(src)
            col.append(t.destination)
            val.append(t.probability)

        # -1 on the diagonal
        row.append(src)
        col.append(src)
        val.append(-1)

        # all-ones column
        row.append(src)
        col.append(n)
        val.append(1)

    Q = scipy.sparse.csr_matrix((val, (row, col)), shape=(n, n + 1))
    QTQ = Q.dot(Q.transpose())
    bQT = np.ones(n)

    v = scipy.sparse.linalg.spsolve(QTQ, bQT)

    assert len(v) == n
    assert math.isclose(sum(v), 1)

    return v

In [6]:
def reward_per_progress(mdp, policy, n_iter=0, eps=0, verbose=False):
    start = time()

    n = mdp.n_states
    reward = np.zeros((2, n), dtype=float)
    progress = np.zeros((2, n), dtype=float)
    prev_rpp = float("-inf")

    ss = steady_state(mdp, policy)

    i = 1
    while True:
        prev = i % 2
        next = (prev + 1) % 2

        for src, actions in enumerate(mdp.tab):
            act = policy[src]

            if act < 0:
                # no action possible in this state
                assert len(actions) == 0
                continue

            for t in actions[act]:
                reward[next, src] += t.probability * (
                    t.reward + reward[prev, t.destination]
                )
                progress[next, src] += t.probability * (
                    t.progress + progress[prev, t.destination]
                )

        steady_reward = np.multiply(ss, reward[next,])
        steady_progress = np.multiply(ss, progress[next,])
        next_rpp = sum(steady_reward) / sum(steady_progress)
        delta = np.abs(prev_rpp - next_rpp)

        if verbose:
            print(f"\riteration {i}: rpp={next_rpp} delta={delta}", end="")

        if n_iter > 0 and i >= n_iter:
            break
        elif eps > 0 and delta < eps:
            break
        else:
            i += 1
            prev_rpp = next_rpp

    if verbose:
        print()  # new line to finish verbose progress bar

    return dict(rpp_iter=i, rpp=next_rpp, rpp_time=time() - start)


for i, a in enumerate(alpha):
    rpp = reward_per_progress(mdp[i], vi[i]["policy"], eps=0.0001)
    print(f"alpha={a} rpp={rpp}")

alpha=0.01 rpp={'rpp_iter': 2, 'rpp': 0.010100010102153764, 'rpp_time': 0.7390792369842529}
alpha=0.05 rpp={'rpp_iter': 2, 'rpp': 0.052631568537553644, 'rpp_time': 0.7956194877624512}
alpha=0.1 rpp={'rpp_iter': 2, 'rpp': 0.11111110842012437, 'rpp_time': 0.6771755218505859}
alpha=0.2 rpp={'rpp_iter': 2, 'rpp': 0.24999998329376974, 'rpp_time': 0.7360646724700928}
alpha=0.3 rpp={'rpp_iter': 2, 'rpp': 0.42854368152419214, 'rpp_time': 0.6683721542358398}
alpha=0.35 rpp={'rpp_iter': 2, 'rpp': 0.5381460991650204, 'rpp_time': 0.8028817176818848}
alpha=0.4 rpp={'rpp_iter': 2, 'rpp': 0.664887995683018, 'rpp_time': 0.6759014129638672}
alpha=0.45 rpp={'rpp_iter': 2, 'rpp': 0.8097040208097033, 'rpp_time': 0.8398287296295166}
alpha=0.49 rpp={'rpp_iter': 2, 'rpp': 0.9197954947739676, 'rpp_time': 0.7351117134094238}
